# Hyperparameter tuning, using all 8 classifiers individually. After choosing the best parameters for each classifier, they will be input into the ensemble models. 

In [1]:
# For model testing purposes, faster runtime for 455 row x 5 columns
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from yellowbrick.classifier import ClassificationReport
from mlxtend.evaluate import bias_variance_decomp
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import time


# iris = datasets.load_breast_cancer()
# X = pd.DataFrame(iris.data[:, :5])  # we only take the first five features.
# y = pd.DataFrame(iris.target)
df_model = pd.DataFrame(pd.read_csv('../2019Data/df_Models2019.csv'))
X = df_model.iloc[:, :-1]
y = df_model.result
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(y_train.unique()) # binary classfication
X_train

[1 0]


,shoot_eff,score_op,off_rtg,def_rtg,sos,ie,efg_pct,to_poss,orb_pct,ft_rate,...,seed_7,seed_8,seed_9,seed_10,seed_11,seed_12,seed_13,seed_14,seed_15,seed_16
487,0.015633,0.037107,5.914737,0.751644,5.163093,3.662162,0.010095,-0.015264,0.041045,-0.016775,...,0,255,1,0,0,0,0,0,0,0
1260,-0.060158,-0.015621,-6.880393,4.813716,-11.694109,-7.771080,-0.033562,-0.000164,-0.021791,0.003040,...,0,0,0,0,0,1,0,0,0,0
695,-0.202263,0.004986,-18.561373,-11.313391,-7.247982,-2.735033,-0.118989,0.035677,0.027175,-0.004217,...,0,255,1,0,0,0,0,0,0,0
541,0.053685,0.033097,8.870478,-0.046278,8.916756,3.153326,0.034807,-0.013180,0.036360,-0.018309,...,0,255,0,0,0,0,0,0,0,0
201,0.046116,0.006299,5.176248,-3.536673,8.712921,7.712511,0.031040,-0.025017,-0.023919,0.007196,...,255,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,0.085489,-0.028070,5.265448,5.891920,-0.626473,-1.291256,0.030945,0.008371,-0.011095,0.087746,...,0,0,0,0,0,0,0,1,0,0
456,0.040818,0.006016,4.499549,7.031982,-2.532432,-2.409015,0.022365,-0.007109,0.009217,-0.028530,...,0,0,0,0,255,0,0,0,0,0
250,-0.139972,0.002839,-13.517829,4.095225,-17.613054,-14.178977,-0.081343,0.003757,-0.045366,0.012992,...,0,0,0,0,1,0,0,0,0,0
360,-0.004794,-0.023225,-3.022612,-4.255995,1.233382,1.446864,0.009357,-0.018519,-0.094538,-0.081276,...,0,0,0,0,0,0,0,0,0,0


In [2]:
def create_pipe(X_train, clf, clf_label='clf'):
    """
    https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf
    Returns an sklearn model pipeline.
    """
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_train.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      (clf_label, clf)])
    return pipe
    
def clfy_report(clf, X_train, X_test, y_train, y_test, param_grid, clf_label='clf', cv=10):
    """
    Tune classifier hyperparameters and print metrics.
    """

    # Create pipeline steps for encoding categorical variables, feature scaling and normalization 
    pipe = create_pipe(X_train, clf, clf_label)
    
    # Instantiate grid search using 10-fold cross validation:
    search = RandomizedSearchCV(pipe, param_grid, cv=cv, n_iter=10)
    
    # Learn relationship between predictors (basketball/tourney features) and outcome,
    # and the best parameters for defining such:
    search.fit(X_train, y_train)
    
    # Predictions on the test set, new data that haven't been introduced to the model:
    predicted = search.predict(X_test)
    
    # Predictions as probabilities:
    probabilities = search.predict_proba(X_test)[:, 1]
    
    # Accuracy scores for the training and test sets:
    train_accuracy = search.score(X_train, y_train)
    test_accuracy = search.score(X_test, y_test)

    print('Best Parameters: {}\n'.format(search.best_params_))
    print('Training Accuracy: {:0.2}'.format(train_accuracy))
    print('Test Accuracy: {:0.2}\n'.format(test_accuracy))
    
    # Confusion matrix labels:
    labels = np.array([['true losses','false wins'], ['false losses','true wins']])
    
    # Model evaluation metrics:
    confusion_mtrx = confusion_matrix(y_test, predicted)
    auc = roc_auc_score(y_test, probabilities)
    fpr, tpr, thresholds = roc_curve(y_test, probabilities)
    logloss = log_loss(y_test, search.predict_proba(X_test))
    
    # Plot all metrics in a grid of subplots:
    fig = plt.figure(figsize=(12, 12))
    grid = plt.GridSpec(2, 4, wspace=0.75, hspace=0.5)
    
    # Top-left plot - confusion matrix:
    plt.subplot(grid[0, :2])
    sns.heatmap(confusion_mtrx, annot=True, fmt="d") #, fmt='')
    plt.xlabel('Predicted Games')
    plt.ylabel('Actual Games');
    
    # Top-right plot - ROC curve:
    plt.subplot(grid[0, 2:])
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, marker='.')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('AUROC: {:0.3}'.format(auc));
    
    # Bottom-left plot - support, or true predictions:
    plt.subplot(grid[1, :2])
    sns.countplot(y=predicted, orient='h')
    plt.yticks([1, 0], ('wins', 'losses'))
    plt.ylabel(''), plt.xlabel('Number Predicted');
    
    # Bottom-right plot - classification report:
    plt.subplot(grid[1, 2:])
    visualizer = ClassificationReport(search, classes=['losses', 'wins'])
    visualizer.fit(X_train, y_train)
    visualizer.score(X_test, y_test)
    g = visualizer.poof();

In [ ]:
classifiers = [
    SVC(probability=True),
    RandomForestClassifier(),
    LogisticRegression(),
    MLPClassifier(max_iter=500), # to allow gaurenteed convergence. 
    GaussianNB(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier()
    ]


# clf__kernel
params=[
    { # SVC
        'clf__kernel': ['rbf', 'linear', 'sigmoid'],
        'clf__C': np.logspace(start=-10, stop=10, num=21) # default 1.0
    },
    { # RandomForestClassifier
        'clf__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'clf__max_features': ['sqrt', 'log2'] # The lower the greater the reduction of variance, but also the greater the increase in bias.
    },
    { # LogisticRegression
        'clf__C': np.logspace(start=-10, stop=10, num=21),
        'clf__penalty': ['none', 'l2']
    },
    { # Neural network multi-layered perceptron, MLPClassifier
      'clf__hidden_layer_sizes': tuple(map(tuple, np.random.randint(low=5, high=20, size=(10, 3)))) # from 5-20 nodes per 3 layers, 10 iterations
    },
    { # GaussianNB
        'clf__var_smoothing': [1e-8, 1e-9, 1e-10]
    },
    { # AdaBoostClassifier
        'clf__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'clf__learning_rate': np.linspace(0.5, 1.5, 10, endpoint=True) # default 1.0
    },
    { # GradientBoostingClassifier
        'clf__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'clf__learning_rate': np.linspace(0.02, 0.18, 9, endpoint=True) # default 0.1
    },
    { # XGBClassifier
#     'clf__learning_rate': np.logspace(start=0.01, stop=0.2, num=10, endpoint = True), # see last example in np.logspace documentation
    'clf__max_depth': [2,3,4,5],
    'clf__booster': ['gbtree', 'gblinear', 'dart']
    }
]

train_acc=[]
test_acc=[]
auc=[]
loggloss=[]
for i, classifier in enumerate(classifiers):
    print(classifier)
    clfy_report(classifier, X_train, X_test, y_train, y_test, param_grid=params[i], cv=5)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


# Ensemble Methods: VotingClassifier, StackingClassifier. 
## 2019 uses same method as 2020

Reminder: High bias implies an underfit model, high variance implies an overfit model. 

## First, a bagging method using a VotingClassifier will be used to reduce the amount of variance in the model. Then, a boosting method using a StackingClassifier will be run comparatively to reduce bias. 

- ensemble averaging methods (VotingClassifier): average predictions to reduce variance. 
- ensemble boosting methods (StackingClassifier): base estimators are built sequentially and one tries to reduce the bias of the combined estimator.
- Bagging methods work best with strong and complex models (e.g. where the data points are unpredictable), in contrast with boosting methods which usually work best with weak models (e.g., shallow decision trees).
- Model with high bias pays very little attention to the training data and oversimplifies the model.
- High variance models pays attention to training data and does not generalize on the data which it hasn’t seen before.



# VotingClassifier

In [ ]:
def create_pipe_ensemble(X_train, ensemble_clf, clf_label='clf'):
    """
    https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf
    Returns an sklearn model pipeline.
    """
    pipe = Pipeline([(clf_label, VotingClassifier(estimators=[
                          (clf_label+str(1), create_pipe(X_train, ensemble_clf.estimators[0][1])),
                          (clf_label+str(2), create_pipe(X_train, ensemble_clf.estimators[1][1])),
                          (clf_label+str(3), create_pipe(X_train, ensemble_clf.estimators[2][1])),
                          (clf_label+str(4), create_pipe(X_train, ensemble_clf.estimators[3][1])),
                          (clf_label+str(5), create_pipe(X_train, ensemble_clf.estimators[4][1])),
                          (clf_label+str(6), create_pipe(X_train, ensemble_clf.estimators[5][1])),
                          (clf_label+str(7), create_pipe(X_train, ensemble_clf.estimators[6][1])),
                          (clf_label+str(8), create_pipe(X_train, ensemble_clf.estimators[7][1]))
                                           ]))])
    return pipe

def clfy_report_ensemble(ensemble_voting, X_train, X_test, y_train, y_test, ensemble_params, cv=10):
    """
    Tune classifier hyperparameters and print metrics.
    return search, train_accuracy, test_accuracy, auc, logloss
    """
    # Instantiate grid search using 10-fold cross validation:
    search = RandomizedSearchCV(ensemble_voting, ensemble_params, cv=cv, n_iter=5)
    
    # Learn relationship between predictors (basketball/tourney features) and outcome,
    # and the best parameters for defining such:
    search.fit(X_train, y_train)
    
    # Predictions on the test set, new data that haven't been introduced to the model:
    predicted = search.predict(X_test)
    
    # Predictions as probabilities:
    probabilities = search.predict_proba(X_test)[:, 1]
    
    # Accuracy scores for the training and test sets:
    train_accuracy = search.score(X_train, y_train)
    test_accuracy = search.score(X_test, y_test)

    print('Best Parameters: {}\n'.format(search.best_params_))
    print('Training Accuracy: {:0.2}'.format(train_accuracy))
    print('Test Accuracy: {:0.2}\n'.format(test_accuracy))
    
    # Confusion matrix labels:
    labels = np.array([['true losses','false wins'], ['false losses','true wins']])
    
    # Model evaluation metrics:
    confusion_mtrx = confusion_matrix(y_test, predicted)
    auc = roc_auc_score(y_test, probabilities)
    fpr, tpr, thresholds = roc_curve(y_test, probabilities)
    logloss = log_loss(y_test, search.predict_proba(X_test))
    
    # Plot all metrics in a grid of subplots:
    fig = plt.figure(figsize=(12, 12))
    grid = plt.GridSpec(2, 4, wspace=0.75, hspace=0.5)
    
    # Top-left plot - confusion matrix:
    plt.subplot(grid[0, :2])
    sns.heatmap(confusion_mtrx, annot=True, fmt="d") #, fmt='')
    plt.xlabel('Predicted Games')
    plt.ylabel('Actual Games');
    
    # Top-right plot - ROC curve:
    plt.subplot(grid[0, 2:])
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, marker='.')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('AUROC: {:0.3}'.format(auc));
    
    # Bottom-left plot - support, or true predictions:
    plt.subplot(grid[1, :2])
    sns.countplot(y=predicted, orient='h')
    plt.yticks([1, 0], ('wins', 'losses'))
    plt.ylabel(''), plt.xlabel('Number Predicted');
    
    # Bottom-right plot - classification report:
    plt.subplot(grid[1, 2:])
    visualizer = ClassificationReport(search, classes=['losses', 'wins'])
    visualizer.fit(X_train, y_train)
    visualizer.score(X_test, y_test)
    g = visualizer.poof();
    
    return search, train_accuracy, test_accuracy, auc, logloss

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)])

# From hyperparameter tuning section above:
# Best Parameters: {'clf__kernel': 'rbf', 'clf__C': 1.0}
# Best Parameters: {'clf__n_estimators': 144, 'clf__max_features': 'log2'}
# Best Parameters: {'clf__penalty': 'l2', 'clf__C': 10000.0}
# Best Parameters: {'clf__hidden_layer_sizes': (5, 18, 8)}
# Best Parameters: {'clf__var_smoothing': 1e-08}
# Best Parameters: {'clf__n_estimators': 177, 'clf__learning_rate': 0.8333333333333333}
# Best Parameters: {'clf__n_estimators': 144, 'clf__learning_rate': 0.04}
# Best Parameters: {'clf__max_depth': 4, 'clf__booster': 'gblinear'}
classifiers = [
    SVC(probability=True, kernel='rbf', C=1.0),         
    RandomForestClassifier(n_estimators=144, max_features='log2'),      
    LogisticRegression(penalty='l2', C=10000.0),          
    MLPClassifier(max_iter=500, hidden_layer_sizes=(5, 18, 8)),   
    GaussianNB(var_smoothing=1e-08),                  
    AdaBoostClassifier(n_estimators=177, learning_rate=0.83),          
    GradientBoostingClassifier(n_estimators=144, learning_rate=0.04), 
    XGBClassifier(max_depth=4, booster='gblinear')             
    ]

pipeline_classifiers=[]
for i in range(0,len(classifiers)):
    pipeline_classifiers.append(Pipeline (steps=[('preprocessor', preprocessor),
        ('clf'+str(i+1), classifiers[i])]))

from sklearn.ensemble import VotingClassifier
ensemble_voting = Pipeline(steps=[('ensemble', VotingClassifier(estimators=[
    ('svc', pipeline_classifiers[0]), 
    ('rdf' , pipeline_classifiers[1]),
    ('lgr' , pipeline_classifiers[2]),
    ('mlp', pipeline_classifiers[3]),
    ('gau', pipeline_classifiers[4]),
    ('ada', pipeline_classifiers[5]), 
    ('gbt', pipeline_classifiers[6]),
    ('xgb', pipeline_classifiers[7])], 
                                voting='soft', 
                                # weights = [1,2,3], 
                                n_jobs=-1))])

import pprint as pp
pp.pprint(sorted(ensemble_voting.get_params().keys())) # used to specify ensemble params, below. 

In [ ]:
ensemble_params=[
    { # SVC
        'ensemble__svc__clf1__kernel': ['rbf', 'linear', 'sigmoid'],
        'ensemble__svc__clf1__C': np.logspace(start=-10, stop=10, num=21) # default 1.0
    },
    { # RandomForestClassifier
        'ensemble__rdf__clf2__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'ensemble__rdf__clf2__max_features': ['sqrt', 'log2'] # The lower the greater the reduction of variance, but also the greater the increase in bias.
    },
    { # LogisticRegression
        'ensemble__lgr__clf3__C': np.logspace(start=-10, stop=10, num=21),
        'ensemble__lgr__clf3__penalty': ['none', 'l2']
    },
    { # Neural network multi-layered perceptron, MLPClassifier
      'ensemble__mlp__clf4__hidden_layer_sizes': tuple(map(tuple, np.random.randint(low=5, high=20, size=(10, 3)))) # from 5-20 nodes per 3 layers, 10 iterations
    },
    { # GaussianNB
        'ensemble__gau__clf5__var_smoothing': [1e-8, 1e-9, 1e-10]
    },
    { # AdaBoostClassifier
        'ensemble__ada__clf6__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'ensemble__ada__clf6__learning_rate': np.linspace(0.5, 1.5, 10, endpoint=True) # default 1.0
    },
    { # GradientBoostingClassifier
        'ensemble__gbt__clf7__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'ensemble__gbt__clf7__learning_rate': np.linspace(0.02, 0.18, 9, endpoint=True) # default 0.1
    },
    { # XGBClassifier
#     'clf__learning_rate': np.logspace(start=0.01, stop=0.2, num=10, endpoint = True), # see last example in np.logspace documentation
    'ensemble__xgb__clf8__max_depth': [2,3,4,5],
    'ensemble__xgb__clf8__booster': ['gbtree', 'gblinear', 'dart']
    }
]
timer_start=time.time()
clfy_report_ensemble(ensemble_voting, X_train, X_test, y_train, y_test, ensemble_params, cv=10)
print(search_ens, train_accuracy, test_accuracy, auc, logloss)
print("Time taken:", round(-(timer_start-time.time()),2),'seconds. ')
del timer_start

# StackingClassifier

In [ ]:
classifiers = [
    ('svc',  SVC(probability=True, kernel='rbf', C=1.0         )),
    ('rdf' , RandomForestClassifier(n_estimators=144, max_features='log2'      )),
    ('lgr' , LogisticRegression(penalty='l2', C=10000.0          )),
    ('mlp',  MLPClassifier(max_iter=500, hidden_layer_sizes=(5, 18, 8)   )),
    ('gau',  GaussianNB(var_smoothing=1e-08)),                  
    ('ada',  AdaBoostClassifier(n_estimators=177, learning_rate=0.83          )),
    ('gbt',  GradientBoostingClassifier(n_estimators=144, learning_rate=0.04) ),
    ('xgb',  XGBClassifier(max_depth=4, booster='gblinear')                      )
    ]

from sklearn.ensemble import StackingClassifier
stack_clf = StackingClassifier(estimators=classifiers, final_estimator=LogisticRegression())
pipe_stack_clf = Pipeline(steps=[('preproc', preprocessor), ('stack', stack_clf)])

In [ ]:
pp.pprint(sorted(pipe_stack_clf.get_params().keys())) # used to specify pipe_stack_clf params, below. 

In [ ]:
pipe_stack_clf_params=[
    { # SVC
        'stack__svc__kernel': ['rbf', 'linear', 'sigmoid'],
        'stack__svc__C': np.logspace(start=-10, stop=10, num=21) # default 1.0
    },
    { # RandomForestClassifier
        'stack__rdf__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'stack__rdf__max_features': ['sqrt', 'log2'] # The lower the greater the reduction of variance, but also the greater the increase in bias.
    },
    { # LogisticRegression
        'stack__lgr__C': np.logspace(start=-10, stop=10, num=21),
        'stack__lgr__penalty': ['none', 'l2']
    },
    { # Neural network multi-layered perceptron, MLPClassifier
        'stack__mlp__hidden_layer_sizes': tuple(map(tuple, np.random.randint(low=5, high=20, size=(10, 3)))) # from 5-20 nodes per 3 layers, 10 iterations
    },
    { # GaussianNB
        'stack__gau__var_smoothing': [1e-8, 1e-9, 1e-10]
    },
    { # AdaBoostClassifier
        'stack__ada__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'stack__ada__learning_rate': np.linspace(0.5, 1.5, 10, endpoint=True) # default 1.0
    },
    { # GradientBoostingClassifier
        'stack__gbt__n_estimators': np.array(np.linspace(100,200,10, endpoint=True) , dtype=np.int32), # default 100
        'stack__gbt__learning_rate': np.linspace(0.02, 0.18, 9, endpoint=True) # default 0.1
    },
    { # XGBClassifier
#     'clf__learning_rate': np.logspace(start=0.01, stop=0.2, num=10, endpoint = True), # see last example in np.logspace documentation
        'stack__xgb__max_depth': [2,3,4,5],
        'stack__xgb__booster': ['gbtree', 'gblinear', 'dart']
    }
]
# print(search, train_accuracy, test_accuracy, auc, logloss)
timer_start=time.time()
search_stk, train_accuracy, test_accuracy, auc, logloss = clfy_report_ensemble(pipe_stack_clf, X_train, X_test, y_train, y_test, pipe_stack_clf_params, cv=10)
print(search_stk, train_accuracy, test_accuracy, auc, logloss)
print("Time taken:", round(-(timer_start-time.time()),2),'seconds. ')
del timer_start

## StackedClassifier loss, bias, variance

In [ ]:
timer_start=time.time()
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        stack_clf, np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), 
        loss='0-1_loss')

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print("Time taken:", round(-(timer_start-time.time()),2),'seconds. ')
del timer_start

## VotingClassifier loss, bias, variance

In [ ]:
timer_start=time.time()
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         ensemble_voting[0], np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), 
    VotingClassifier(estimators=[
        classifiers[0], 
        classifiers[1],
        classifiers[2],
        classifiers[3],
        classifiers[4],
        classifiers[5], 
        classifiers[6],
        classifiers[7]], 
                        voting='soft', 
                        # weights = [1,2,3], 
                        n_jobs=-1),
        np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), 
        loss='0-1_loss')

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print("Time taken:", round(-(timer_start-time.time()),2),'seconds. ')
del timer_start

- ensemble averaging methods (VotingClassifier): average predictions to reduce variance. 
- ensemble boosting methods (StackingClassifier): base estimators are built sequentially and one tries to reduce the bias of the combined estimator.
- Bagging methods work best with strong and complex models (e.g. where the data points are unpredictable), in contrast with boosting methods which usually work best with weak models (e.g., shallow decision trees).
- Model with high bias pays very little attention to the training data and oversimplifies the model.
- High variance models pays attention to training data and does not generalize on the data which it hasn’t seen before.



# Results:

#### From these results, the both of the VotingClassifier and StackingClassifier models built have:
- very low variance  (~0.10 - 0.14)
- low-moderate loss (~0.31 - 0.34)
- low-moderate bias (~0.30 - 0.32)  </ul>
The results above implies that the final model needs to use the StackingClassifier, to lower bias. However, the 10-fold cross-validated accuracy is around the same for the VotingClassifier (Training Accuracy: 0.82, Test Accuracy: 0.69) compared to the StackingClassifier (Training Accuracy: 0.64, Test Accuracy: 0.7). 
### Therefore, either VotingClassifier or StackingClassifier could be used as the primary prediction model.

# Create a bracket using both methods. 

In [ ]:
df_predict = pd.read_csv('../2019Data/SampleSubmissionStage2.csv')
df_predict.head()

In [ ]:
def get_year_team1_team2(ID):
    """Return a tuple with the year, team1 and team2
    for each ID in the sample submission file of possible matches."""
    return (int(x) for x in ID.split('_'))

In [ ]:
df_features = pd.read_csv('../2019Data/df_features.csv')
diff = []
data = []

for i, row in df_predict.iterrows():

    year, team1, team2 = get_year_team1_team2(row.ID)

    # Save 2018 stats/features for the first ID:
    team1 = df_features[(df_features['Season'] == year) & (df_features['TeamID'] == team1)].values[0]

    # Save 2018 stats/features for the first ID:
    team2 = df_features[(df_features['Season'] == year) & (df_features['TeamID'] == team2)].values[0]

    diff = team1 - team2

    data.append(diff)

n_poss_games = len(df_predict)
columns = df_features.columns.get_values()
final_predictions = pd.DataFrame(np.array(data).reshape(n_poss_games, np.array(data).shape[1]), columns=(columns))
final_predictions.drop(['Season', 'TeamID'], inplace=True, axis=1)

# StackingClassifier Bracket Predictions

In [ ]:
predictions = search_stk.predict_proba(final_predictions)[:, 1]
clipped_predictions = np.clip(predictions, 0.05, 0.95)
df_predict.Pred = clipped_predictions
df_predict.to_csv('search_stk.csv', index=False)

from bracketeer import build_bracket
b = build_bracket(
        outputPath='search_stk.png', # in /Ryan
        submissionPath='search_stk.csv',
        teamsPath='../2020Data/MDataFiles_Stage1/MTeams.csv',
        seedsPath='../2020Data/MDataFiles_Stage1/MNCAATourneySeeds.csv',
        slotsPath='../2020Data/MDataFiles_Stage1/MNCAATourneySlots.csv',
        year=2019
)
from IPython.display import Image
Image(filename='search_stk.png')

# VotingClassifier Bracket Predictions

In [ ]:
predictions = search_ens.predict_proba(final_predictions)[:, 1]
clipped_predictions = np.clip(predictions, 0.05, 0.95)
df_predict.Pred = clipped_predictions
df_predict.to_csv('search_ens.csv', index=False)

from bracketeer import build_bracket
b = build_bracket(
        outputPath='search_ens.png', # in /Ryan
        submissionPath='search_ens.csv',
        teamsPath='../2020Data/MDataFiles_Stage1/MTeams.csv',
        seedsPath='../2020Data/MDataFiles_Stage1/MNCAATourneySeeds.csv',
        slotsPath='../2020Data/MDataFiles_Stage1/MNCAATourneySlots.csv',
        year=2019
)
from IPython.display import Image
Image(filename='search_ens.png')